## Using SeqTag model to classify errors in MT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tqdm
import os
os.getcwd()

'/content'

In [3]:
os.chdir('/content/drive/MyDrive/thesis') 
os.getcwd()

'/content/drive/MyDrive/thesis'

In [4]:
!git clone https://github.com/Unbabel/COMET

In [22]:
!pip install pyyaml
!pip install scipy
!pip install transformers
!pip install entmax
!pip install sentencepiece
!pip install torch
!pip install pytorch_lightning==2.0.1
!pip install huggingface_hub 

### Read and prepare the test set for scoring

In [2]:
import pandas as pd
import time
import glob
import re
import os
import numpy as np
import sys
import string

from tqdm.notebook import trange, tqdm

from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None)

In [ ]:
PROJECT_DIR = "segments"
SCORES_DIR = "scores"

SOURCE_FILE = "segments_test_set.xlsx"
RESULT_FILE = f"SCORES_{SOURCE_FILE}"

SOURCE_LANG = 'en'
TARGET_LANG = 'ru'

In [8]:
segments_df = pd.read_excel(SOURCE_FILE)
segments_df.head(1)

,source,reference,mt
0,"OK, we have three of three NFL players here.","Итак, у нас здесь три из трёх игроков НФЛ.","Итак, у нас здесь трое из трех игроков NFL."
1,"I mean, describe to us this Bullrun program where it's alleged that the NSA specifically weakened security in order to get the type of access that you've spoken of.","Расскажите нам о программе Bullrun, согласно которой АНБ якобы преднамеренно ослабляло безопасность коммуникаций, чтобы получить тот уровень доступа, о котором Вы говорите.","Я имею в виду, опишите нам эту программу Буллран, в которой утверждается, что АНБ специально ослабило безопасность, чтобы получить тот тип доступа, о котором вы говорили."
2,We live in a world where GDP is the benchmark of success in a global economy.,"Мы живём в мире, где ВВП стал показателем успеха в мировой экономики.","Мы живем в мире, где GDP является эталоном успеха в глобальной экономике."
3,"Okay, so the ALS Association, they think by year end, it'll be 160 million dollars.","Ассоциация БАС считает, что к концу года будет собрано 160 млн долларов.","Итак, Ассоциация ALS, по их мнению, к концу года получит 160 миллионов долларов."
4,Each person must know their HIV status.,Каждый человек должен знать свой ВИЧ-статус.,Каждый человек должен знать свой HIV-статус.


In [10]:
os.chdir('/content/drive/MyDrive/thesis/COMET') 
os.getcwd()

'/content/drive/MyDrive/thesis/COMET'

In [11]:
import torch

In [12]:
num_cores = 8
torch.set_num_threads(num_cores)

In [14]:
MODEL_AS_METRIC = 'wmt22-seqtag-mqm'

Testing SeqTag on a couple of sentences

In [23]:
from comet import download_model, load_from_checkpoint

In [24]:
model = load_from_checkpoint('/content/drive/MyDrive/thesis/COMET/wmt22-seqtag-mqm/checkpoints/model.ckpt')

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.6.4 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file wmt22-seqtag-mqm/checkpoints/model.ckpt`


In [25]:
data = [
    {
        "src": "Dem Feuer konnte Einhalt geboten werden",
        "mt": "The fire could be stopped",
        "ref": "They were able to control the fire."
    },
    {
        "src": "Schulen und Kindergärten wurden eröffnet.",
        "mt": "Schools and kindergartens were open",
        "ref": "Schools and kindergartens opened"
    }
]

In [27]:
model_output = model.predict(data, batch_size=8, gpus=1)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.15it/s]


In [28]:
model_output

Prediction([('scores', [0.6787528991699219, 0.7925233840942383]),
            ('system_score', 0.7356381416320801),
            ('metadata',
             Prediction([('src_scores',
                          [0.6785367727279663, 0.7679629325866699]),
                         ('ref_scores',
                          [0.7040753364562988, 0.8182905912399292]),
                         ('unified_scores',
                          [0.6536464691162109, 0.7913165092468262]),
                         ('subword_scores',
                          [[('▁The', 0.34047579765319824),
                            ('▁fire', 0.5225199460983276),
                            ('▁could', 0.4745945930480957),
                            ('▁be', 0.4918487071990967),
                            ('▁stopped', 0.6508753299713135)],
                           [('▁School', 0.2730139493942261),
                            ('s', 0.29079359769821167),
                            ('▁and', 0.36390990018844604),
          

In [29]:
def comet_as_metric_func(
    dataframe: pd.core.frame.DataFrame,  # type: ignore,
    col_name_source: str,
    col_name_hyp: str,
    col_name_ref: str,
    use_model: str,
    gpus: int,
) -> pd.core.frame.DataFrame:  # type: ignore

    list_source = dataframe[col_name_source].to_list()
    list_hyp = dataframe[col_name_hyp].to_list()
    list_ref = dataframe[col_name_ref].to_list()

    # load all

    data = {"src": list_source, "mt": list_hyp, "ref": list_ref}
    data = [dict(zip(data, t)) for t in zip(*data.values())]

    from comet import download_model, load_from_checkpoint

    model = load_from_checkpoint('/content/drive/MyDrive/thesis/COMET/wmt22-seqtag-mqm/checkpoints/model.ckpt')

    model_output = model.predict(
        data, gpus=gpus
    )
    word_scores = model_output.metadata.subword_scores
    word_scores_sorted = []
    for word_score in word_scores:
        word_score = sorted(word_score,key=lambda x: x[1],reverse=True)[:20]
        word_scores_sorted.append(word_score)
    
    scores_list, corpus_score = model_output.scores, model_output.system_score
    
    name_score = MODEL_AS_METRIC
    dataframe[name_score] = scores_list
    
    dataframe['corpus score'] = corpus_score
    
    dataframe['word_scores'] = word_scores
    
    dataframe['errors'] = word_scores_sorted

    return dataframe


In [30]:
segments_df = comet_as_metric_func(segments_df, 
                             col_name_source = 'source', 
                             col_name_hyp = 'mt', 
                             col_name_ref = 'reference',
                             use_model = MODEL_AS_METRIC,
                             gpus = 1)

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.6.4 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file wmt22-seqtag-mqm/checkpoints/model.ckpt`
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 25/25 [00:21<00:00,  1.17it/s]


In [31]:
segments_df.head(10)

,source,reference,mt,reason,wmt22-seqtag-mqm,corpus score,word_scores,errors
0,"OK, we have three of three NFL players here.","Итак, у нас здесь три из трёх игроков НФЛ.","Итак, у нас здесь трое из трех игроков NFL.",acronyms,0.601369,-0.311334,"[(▁Итак, 0.1069316416978836), (,, 0.07770725339651108), (▁у, 0.09060796350240707), (▁нас, 0.08213721960783005), (▁здесь, 0.07255853712558746), (▁тро, 0.057215042412281036), (е, 0.10864969342947006), (▁из, 0.05544544383883476), (▁трех, 0.042844563722610474), (▁игроков, 0.06365104019641876), (▁NFL, 0.1273880898952484), (., 0.07495640963315964)]","[(▁NFL, 0.1273880898952484), (е, 0.10864969342947006), (▁Итак, 0.1069316416978836), (▁у, 0.09060796350240707), (▁нас, 0.08213721960783005), (,, 0.07770725339651108), (., 0.07495640963315964), (▁здесь, 0.07255853712558746), (▁игроков, 0.06365104019641876), (▁тро, 0.057215042412281036), (▁из, 0.05544544383883476), (▁трех, 0.042844563722610474)]"
1,"I mean, describe to us this Bullrun program where it's alleged that the NSA specifically weakened security in order to get the type of access that you've spoken of.","Расскажите нам о программе Bullrun, согласно которой АНБ якобы преднамеренно ослабляло безопасность коммуникаций, чтобы получить тот уровень доступа, о котором Вы говорите.","Я имею в виду, опишите нам эту программу Буллран, в которой утверждается, что АНБ специально ослабило безопасность, чтобы получить тот тип доступа, о котором вы говорили.",acronyms,0.045041,-0.311334,"[(▁Я, 0.1389012187719345), (▁имею, 0.1648944914340973), (▁в, 0.05742048844695091), (▁виду, 0.13678967952728271), (,, 0.18801835179328918), (▁о, 0.20019179582595825), (пишите, 0.13455577194690704), (▁нам, 0.0882662758231163), (▁эту, 0.026207810267806053), (▁программу, 0.08424928784370422), (▁Бул, 0.6619203090667725), (л, 0.8324640989303589), (ран, 0.8820561170578003), (,, 0.1728247106075287), (▁в, 0.2123938500881195), (▁которой, 0.15930697321891785), (▁утверждает, 0.161564901471138), (ся, 0.08522588014602661), (,, 0.044819872826337814), (▁что, 0.017054015770554543), (▁АН, 0.060748156160116196), (Б, 0.03693024069070816), (▁специально, 0.047607116401195526), (▁о, 0.06707533448934555), (слаб, 0.06108957529067993), (ило, 0.040477968752384186), (▁безопасность, 0.06101026386022568), (,, 0.03389894217252731), (▁чтобы, 0.012669738382101059), (▁получить, 0.02784055843949318), (▁тот, 0.03133223578333855), (▁тип, 0.09789715707302094), (▁доступа, 0.07594802975654602), (,, 0.02961190603673458), (▁о, 0.03749142587184906), (▁котором, 0.034058380872011185), (▁вы, 0.11905922740697861), (▁говорили, 0.11533006280660629), (., 0.13715681433677673)]","[(ран, 0.8820561170578003), (л, 0.8324640989303589), (▁Бул, 0.6619203090667725), (▁в, 0.2123938500881195), (▁о, 0.20019179582595825), (,, 0.18801835179328918), (,, 0.1728247106075287), (▁имею, 0.1648944914340973), (▁утверждает, 0.161564901471138), (▁которой, 0.15930697321891785), (▁Я, 0.1389012187719345), (., 0.13715681433677673), (▁виду, 0.13678967952728271), (пишите, 0.13455577194690704), (▁вы, 0.11905922740697861), (▁говорили, 0.11533006280660629), (▁тип, 0.09789715707302094), (▁нам, 0.0882662758231163), (ся, 0.08522588014602661), (▁программу, 0.08424928784370422)]"
2,We live in a world where GDP is the benchmark of success in a global economy.,"Мы живём в мире, где ВВП стал показателем успеха в мировой экономики.","Мы живем в мире, где GDP является эталоном успеха в глобальной экономике.",acronyms,0.579424,-0.311334,"[(▁Мы, 0.012581933289766312), (▁живе, 0.009849824011325836), (м, 0.012900669127702713), (▁в, 0.022077325731515884), (▁мире, 0.010033882223069668), (,, 0.013114986941218376), (▁где, 0.011364590376615524), (▁, 0.1476125717163086), (GDP, 0.3069528043270111), (▁является, 0.06641090661287308), (▁эта, 0.11156110465526581), (ло, 0.1742878258228302), (ном, 0.06380042433738708), (▁успеха, 0.024717746302485466), (▁в, 0.02475670538842678), (▁глобальн, 0.026891272515058517), (ой, 0.07394921779632568), (▁экономике, 0.02103700116276741), (., 0

In [32]:
segments_df.shape

(400, 8)

In [33]:
segments_df.drop(columns='errors', inplace=True)

In [35]:
segments_df.to_excel(RESULT_FILE, index=False)